In [2]:
! pip install transformers
! pip install torch

### importing libraries 

In [3]:
import torch
import numpy as np
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration
import json
from itertools import zip_longest

### defining general variables

In [4]:
#defining general valriables throughout the whole notebook
EPOCH = 10
batch_size = 16
max_input_length = 64
max_label_length = 8
MODEL_LINK = "google/flan-t5-small"

In [5]:
tokenizer = T5Tokenizer.from_pretrained(MODEL_LINK, legacy=False)
model = T5ForConditionalGeneration.from_pretrained(MODEL_LINK).to('cuda')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### reading dataset and cleaning it

In [6]:
# a funciton to read data off of a database link

def get_data(address):
    lines = []
    with open(address) as file:
        for line in file:
            x = json.loads(line)
            lines.append(x)
    sentences, orl, sep_sentences = [], [], []
    for i in range(len(lines)):
        sep_sentences.append(lines[i]['sentences'])
        sentences.append(' '.join(lines[i]['sentences']))
        orl.append(lines[i]['orl'])
    dataframe = pd.DataFrame({'sentence': sentences, 'orl': orl, 'sep_sent': sep_sentences})
    return dataframe

### getting training data into df and dividing each of {agent, target, dse} elements

In [7]:
df = get_data("/kaggle/input/ds-json-format/json_format_dataset/0/aaai19srl.train0.conll.json")
df

,sentence,orl,sep_sent
0,The Kimberley Provincial Hospital said it woul...,"[[6, 8, 0, 3, AGENT], [6, 8, 6, 8, DSE], [6, 8...","[The, Kimberley, Provincial, Hospital, said, i..."
1,Saeed said indications were that those tests w...,"[[1, 1, 0, 0, AGENT], [1, 1, 1, 1, DSE], [1, 1...","[Saeed, said, indications, were, that, those, ..."
2,He said it was his opinion that the patient --...,"[[4, 5, 0, 0, AGENT], [4, 5, 4, 5, DSE], [4, 5...","[He, said, it, was, his, opinion, that, the, p..."
3,The woman was admitted to the hospital on Satu...,"[[10, 10, 0, 1, AGENT], [10, 10, 10, 10, DSE],...","[The, woman, was, admitted, to, the, hospital,..."
4,`` Since our technical equipment is far from p...,"[[22, 22, 2, 4, TARGET], [22, 22, 10, 10, TARG...","[``, Since, our, technical, equipment, is, far..."
...,...,...,...
2444,Benjamin Franklin Federal Savings & Loan Assoc...,"[[9, 9, 8, 8, AGENT], [9, 9, 9, 9, DSE]]","[Benjamin, Franklin, Federal, Savings, &, Loan..."
2445,thrift said the restructuring should help it m...,"[[1, 1, 1, 1, DSE], [1, 1, 2, 3, TARGET]]","[thrift, said, the, restructuring, should, hel..."
2446,Details of the restructuring wo n't be made fi...,"[[11, 11, 10, 10, AGENT], [11, 11, 11, 11, DSE]]","[Details, of, the, restructuring, wo, n't, be,..."
2447,"Jay Stevens , an analyst with Dean Witter Reyn...","[[12, 12, 11, 11, AGENT], [12, 12, 12, 12, DSE]]","[Jay, Stevens, ,, an, analyst, with, Dean, Wit..."


In [8]:
# Function to organize the data into separate columns

def organize_data(atributs, sentence):
    AGENT, DSE, TARGET = '', '', ''
    for sublist in atributs:
        if sublist[-1] == 'AGENT':
            start = int(sublist[2])
            end = int(sublist[3] + 1)
            AGENT += ' '.join(sentence[start:end]) + '|'
        elif sublist[-1] == 'DSE':
            start = int(sublist[0])
            end = int(sublist[1] + 1)
            DSE += ' '.join(sentence[start:end]) + '|'
        elif sublist[-1] == 'TARGET':
            start = int(sublist[2])
            end = int(sublist[3] + 1)
            TARGET += ' '.join(sentence[start:end]) + '|'
    return AGENT, DSE, TARGET

# Organize tarin data into diffrent columns
for i in range(len(df)):
    agent, dse, target = organize_data(df['orl'][i], df['sep_sent'][i])
    df.loc[i, 'agent'] = agent
    df.loc[i, 'dse'] = dse
    df.loc[i, 'target'] = target
df.head(-10)

,sentence,orl,sep_sent,agent,dse,target
0,The Kimberley Provincial Hospital said it woul...,"[[6, 8, 0, 3, AGENT], [6, 8, 6, 8, DSE], [6, 8...","[The, Kimberley, Provincial, Hospital, said, i...",The Kimberley Provincial Hospital|,would probably know|,whether one of its patients had Congo Fever|
1,Saeed said indications were that those tests w...,"[[1, 1, 0, 0, AGENT], [1, 1, 1, 1, DSE], [1, 1...","[Saeed, said, indications, were, that, those, ...",Saeed|,said|,those tests|
2,He said it was his opinion that the patient --...,"[[4, 5, 0, 0, AGENT], [4, 5, 4, 5, DSE], [4, 5...","[He, said, it, was, his, opinion, that, the, p...",He|,his opinion|,the patient -- a woman|
3,The woman was admitted to the hospital on Satu...,"[[10, 10, 0, 1, AGENT], [10, 10, 10, 10, DSE],...","[The, woman, was, admitted, to, the, hospital,...",The woman|,complaining|,severe joint pains|
4,`` Since our technical equipment is far from p...,"[[22, 22, 2, 4, TARGET], [22, 22, 10, 10, TARG...","[``, Since, our, technical, equipment, is, far...",Nazarov|,said|,our technical equipment|we|
...,...,...,...,...,...,...
2434,"The investor , Donald A. Wright , plans to run...","[[7, 7, 0, 6, AGENT], [7, 7, 7, 7, DSE], [7, 7...","[The, investor, ,, Donald, A., Wright, ,, plan...","The investor , Donald A. Wright ,|",plans|,to run the company|
2435,The transaction has been approved by Kyle 's b...,"[[2, 4, 0, 1, TARGET], [2, 4, 2, 4, DSE], [2, ...","[The, transaction, has, been, approved, by, Ky...",Kyle 's board|the company 's shareholders|,has been approved|approval|,The transaction|
2436,holders adopted a shareholder-rights plan at t...,"[[1, 1, 1, 1, DSE], [1, 1, 2, 4, TARGET]]","[holders, adopted, a, shareholder-rights, plan...",,adopted|,a shareholder-rights plan|
2437,"Earlier this month , the company said it expec...","[[8, 8, 7, 7, AGENT], [8, 8, 8, 8, DSE], [8, 8...","[Earlier, this, month, ,, the, company, said, ...",it|,expected|,to break even for the quarter|


In [9]:
# for i in range(len(df)):
#     if (int(len(df.iloc[i]['orl'])) >= 6):
#         print(i)


print(df.iloc[2430]['sentence'])
print(df.iloc[2430]['orl'])
print(df.iloc[2430])

While bond investors would have preferred growth to be a little slower , they were cheered by inflation measures in the data that showed prices rising at a modest annual rate of 2.9 % .
[[3, 5, 1, 2, 'AGENT'], [3, 5, 3, 5, 'DSE'], [3, 5, 6, 11, 'TARGET'], [14, 15, 13, 13, 'AGENT'], [14, 15, 14, 15, 'DSE'], [14, 15, 17, 18, 'TARGET']]
sentence    While bond investors would have preferred grow...
orl         [[3, 5, 1, 2, AGENT], [3, 5, 3, 5, DSE], [3, 5...
sep_sent    [While, bond, investors, would, have, preferre...
agent                                    bond investors|they|
dse                        would have preferred|were cheered|
target       growth to be a little slower|inflation measures|
Name: 2430, dtype: object


### getting evaluation data ready into dev_df and dividing elements into columns

In [9]:
dev_df = get_data("/kaggle/input/ds-json-format/json_format_dataset/0/aaai19srl.dev0.conll.json")
dev_df

,sentence,orl,sep_sent
0,The owner though that the animal was suffering...,"[[2, 2, 0, 1, AGENT], [2, 2, 2, 2, DSE], [2, 2...","[The, owner, though, that, the, animal, was, s..."
1,"The owner put down the animal , although the v...","[[10, 11, 8, 9, AGENT], [10, 11, 10, 11, DSE],...","[The, owner, put, down, the, animal, ,, althou..."
2,"GATUNA , Rwanda , July 6 -LRB- AFP -RRB- - Pre...","[[30, 32, 30, 32, DSE], [30, 32, 33, 35, TARGET]]","[GATUNA, ,, Rwanda, ,, July, 6, -LRB-, AFP, -R..."
3,"The formerly close allies fell out in 1999 , t...","[[4, 5, 0, 3, AGENT], [4, 5, 4, 5, DSE], [4, 5...","[The, formerly, close, allies, fell, out, in, ..."
4,"In March , Uganda declared Rwanda a hostile na...","[[4, 4, 3, 3, AGENT], [4, 4, 4, 4, DSE], [4, 4...","[In, March, ,, Uganda, declared, Rwanda, a, ho..."
...,...,...,...
1033,He said the independent power segment could gr...,"[[1, 1, 0, 0, AGENT], [1, 1, 1, 1, DSE], [1, 1...","[He, said, the, independent, power, segment, c..."
1034,Westinghouse also expects its international sa...,"[[2, 2, 0, 0, AGENT], [2, 2, 2, 2, DSE]]","[Westinghouse, also, expects, its, internation..."
1035,Analysts had expected Consolidated to post a s...,"[[2, 2, 0, 0, AGENT], [2, 2, 2, 2, DSE], [2, 2...","[Analysts, had, expected, Consolidated, to, po..."
1036,"They have to continue to tighten their belts ,...","[[10, 10, 0, 0, TARGET], [10, 10, 10, 10, DSE]...","[They, have, to, continue, to, tighten, their,..."


In [10]:
for i in range(len(dev_df)):
    agent, dse, target = organize_data(dev_df['orl'][i], dev_df['sep_sent'][i])
    dev_df.loc[i, 'agent'] = agent
    dev_df.loc[i, 'dse'] = dse
    dev_df.loc[i, 'target'] = target
dev_df.head(10)

,sentence,orl,sep_sent,agent,dse,target
0,The owner though that the animal was suffering...,"[[2, 2, 0, 1, AGENT], [2, 2, 2, 2, DSE], [2, 2...","[The, owner, though, that, the, animal, was, s...",The owner|,though|,the animal|
1,"The owner put down the animal , although the v...","[[10, 11, 8, 9, AGENT], [10, 11, 10, 11, DSE],...","[The, owner, put, down, the, animal, ,, althou...",the vet|,had forbidden|,him to do so|
2,"GATUNA , Rwanda , July 6 -LRB- AFP -RRB- - Pre...","[[30, 32, 30, 32, DSE], [30, 32, 33, 35, TARGET]]","[GATUNA, ,, Rwanda, ,, July, 6, -LRB-, AFP, -R...",,soured relations between|,their neighbouring countries|
3,"The formerly close allies fell out in 1999 , t...","[[4, 5, 0, 3, AGENT], [4, 5, 4, 5, DSE], [4, 5...","[The, formerly, close, allies, fell, out, in, ...",The formerly close allies|The formerly close a...,fell out|mounting rivalry|,each|
4,"In March , Uganda declared Rwanda a hostile na...","[[4, 4, 3, 3, AGENT], [4, 4, 4, 4, DSE], [4, 4...","[In, March, ,, Uganda, declared, Rwanda, a, ho...",Uganda|,declared|alleged|support|,Rwanda|Kigali|a rival to Museveni in a preside...
5,The two countries have each accused the other ...,"[[3, 5, 3, 5, DSE], [3, 5, 6, 7, TARGET], [9, ...","[The, two, countries, have, each, accused, the...",,have each accused|backing|,the other|dissidents|
6,"According to military experts , it is possible...","[[0, 1, 0, 1, DSE], [0, 1, 2, 3, AGENT], [0, 1...","[According, to, military, experts, ,, it, is, ...",military experts|,According to|,clashes will resume between the Taleban and UI...
7,`` We hope that this will serve as an occasion...,"[[2, 2, 1, 1, AGENT], [2, 2, 2, 2, DSE], [2, 2...","[``, We, hope, that, this, will, serve, as, an...",We|,hope|,this will serve as an occasion for LG to make ...
8,The navy craft had approached the small boat t...,"[[8, 8, 8, 8, DSE], [8, 8, 9, 9, TARGET]]","[The, navy, craft, had, approached, the, small...",,thinking|,it|
9,Merchant vessels have been warned to operate s...,"[[2, 4, 0, 1, TARGET], [2, 4, 2, 4, DSE], [2, ...","[Merchant, vessels, have, been, warned, to, op...",,have been warned|,Merchant vessels|being detected by rebels|


In [10]:
df.drop(columns='sep_sent')

,sentence,orl,agent,dse,target
0,The Kimberley Provincial Hospital said it woul...,"[[6, 8, 0, 3, AGENT], [6, 8, 6, 8, DSE], [6, 8...",The Kimberley Provincial Hospital|,would probably know|,whether one of its patients had Congo Fever|
1,Saeed said indications were that those tests w...,"[[1, 1, 0, 0, AGENT], [1, 1, 1, 1, DSE], [1, 1...",Saeed|,said|,those tests|
2,He said it was his opinion that the patient --...,"[[4, 5, 0, 0, AGENT], [4, 5, 4, 5, DSE], [4, 5...",He|,his opinion|,the patient -- a woman|
3,The woman was admitted to the hospital on Satu...,"[[10, 10, 0, 1, AGENT], [10, 10, 10, 10, DSE],...",The woman|,complaining|,severe joint pains|
4,`` Since our technical equipment is far from p...,"[[22, 22, 2, 4, TARGET], [22, 22, 10, 10, TARG...",Nazarov|,said|,our technical equipment|we|
...,...,...,...,...,...
2444,Benjamin Franklin Federal Savings & Loan Assoc...,"[[9, 9, 8, 8, AGENT], [9, 9, 9, 9, DSE]]",it|,plans|,
2445,thrift said the restructuring should help it m...,"[[1, 1, 1, 1, DSE], [1, 1, 2, 3, TARGET]]",,said|,the restructuring|
2446,Details of the restructuring wo n't be made fi...,"[[11, 11, 10, 10, AGENT], [11, 11, 11, 11, DSE]]",regulators|,approve|,
2447,"Jay Stevens , an analyst with Dean Witter Reyn...","[[12, 12, 11, 11, AGENT], [12, 12, 12, 12, DSE]]",he|,expected|,


In [11]:
def pipeDivider(pipedString):
    listOfItems = []
    listOfItems = pipedString.split('|')[:-1]
    return listOfItems

In [12]:
df['prompt'] = df.apply(lambda x: ['sentence is: ' + x.sentence] + ['find target for this dse:' + i for i in pipeDivider(str(x.dse))], axis=1)

In [14]:
print(df.iloc[7]['prompt'])

['sentence is: The Nouvel Observateur said that Beijing is unlikely to annex Taiwan by force , despite its deployment of missiles along its coastal provinces opposite Taiwan , but is instead resorting to smarter tactics , bringing the Taiwan authorities to heel through Taiwan businessmen who are eager to explore the mainland Chinese market .', 'find target for this dse:said', 'find target for this dse:are eager']


In [15]:
from torch.utils.data import DataLoader, Dataset
# Define a custom dataset
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length4text, max_length4label):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length4text = max_length4text
        self.max_length4label = max_length4label

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data['sentence'][idx]
        agent = self.data['agent'][idx]
        dse = self.data['dse'][idx]
        target = self.data['target'][idx]
        text_encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length4text, return_tensors='pt').to("cuda")
        #agent
        agent_encoding = self.tokenizer(agent, truncation=True, padding='max_length', max_length=self.max_length4label, return_tensors='pt').to("cuda")
        #dse
        dse_encoding = self.tokenizer(dse, truncation=True, padding='max_length', max_length=self.max_length4label, return_tensors='pt').to("cuda")
        #target
        target_encoding = self.tokenizer(target, truncation=True, padding='max_length', max_length=self.max_length4label, return_tensors='pt').to("cuda")
        return {
            #text
            'input_ids': text_encoding['input_ids'].squeeze(),
            'attention_mask': text_encoding['attention_mask'].squeeze(),
            #agent
            'agent_id': agent_encoding['input_ids'].squeeze(),
            'agent_mask': agent_encoding['attention_mask'].squeeze(),
            #dse
            'dse_id': dse_encoding['input_ids'].squeeze(),
            'dse_mask': dse_encoding['attention_mask'].squeeze(),
            #target
            'target_id': target_encoding['input_ids'].squeeze(),
            'target_mask': target_encoding['attention_mask'].squeeze()
        }

In [16]:
# Create a data loader
train_dataset = CustomDataset(df[:2500], tokenizer, max_length4text=max_input_length, max_length4label=max_label_length)
train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = CustomDataset(dev_df, tokenizer, max_length4text=max_input_length, max_length4label=max_label_length)
val_data_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Define the loss function and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [17]:
def cleaner(str_item):
    final_string_list = []
    string_list = str_item.split('|')
    for i in string_list:
        j = i.replace(' ', '')
        if len(j) > 0:
            # data cleaning
            final_string_list.append(j)
            
    return final_string_list


In [18]:
cleaner("a|b|  |")

['a', 'b']

In [19]:
def f1_calculator(pred_list, actual_list):
    cleaned_pred, cleaned_actual = [], []
    matched = 0
    for i in range(len(pred_list)):
        pred_items = cleaner(pred_list[i])
        actual_items = cleaner(actual_list[i])
        # function:
        for j in range(len(actual_items)):
            print(actual_items[j])
            if actual_items[j] in pred_items:
                matched += 1
                print(f'** actual ** \n')
        cleaned_pred.extend(pred_items)
        cleaned_actual.extend(actual_items)
    
    prediction_len = len(cleaned_pred)
    actual_len = len(cleaned_actual)
    print(f'matched: {matched}, prediction_len:{prediction_len}, actual_len:{actual_len} \n')
    try:
        precision = (matched / prediction_len)
        recall = (matched / actual_len)
        f1 = (2 * (precision * recall)) / (precision + recall)
    except:
        f1 = 0
    return f1

In [1]:
def evaluate_model(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        true_counted, prediction_counted, actual_counted = 0,0,0
        matched, prediction_len, actual_len = 0,0,0
        actual_list, prediction_list, f1 = [], [], []
        for batch_idx, batch in enumerate(dataloader):
            # Move data to the specified device
            # batch = {key: value.to('cuda') for key, value in batch.items()}

            # Forward pass
            ids = batch['input_ids']
            mask = batch['attention_mask']
            target_id = batch['dse_id']
            
            actuals = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in target_id]
            
            output = generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=64, 
              )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in output]
            
            actual_list.extend(actuals)
            prediction_list.extend(preds)
    return f1_calculator(prediction_list, actual_list)

In [ ]:
model.train()
for epoch in range(EPOCH):
    losses = []
    print(f'epoch: {epoch} \n')
    for batch in train_data_loader:
        inputs = batch['input_ids']
        attention_mask = batch['attention_mask']
        dse_id = batch['dse_id']
        dse_mask = batch['dse_mask']

        optimizer.zero_grad()
        outputs = model(inputs, attention_mask=attention_mask, labels=dse_id)
        loss = outputs.loss
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
    f1 = evaluate_model(model, val_data_loader)
    print(f'loss: {np.mean(losses)}, f1 validation:{f1} \n end of epoch{epoch}. \n')

In [ ]:
for epoch in range(EPOCH):
    print(f'epoch: {epoch} \n')
    accuracy = evaluate_model(model, val_data_loader)
    print(f'accuracy: {accuracy}')

In [ ]:
input_text = "find expression of the sentence: The Palestinians want nothing from Washington but to understand their cause and stand beside right and justice ."
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))